In [3]:
# Python Program to Pull Historical Crypto Price Data from FTX API
# All FTX Markets shown here: https://ftx.com/markets
# 5 Coins Used for Program: BTC, ETH, SOL, XRP, MATIC

In [4]:
# Required libaries to run FTX API program

import pandas as pd
import os
import json
import sys
import time
import hmac
import requests
import hvplot.pandas
from ftx_functions import (get_historical_data, get_summary_stats, get_average_spread)

In [5]:
# Pulls HISTORICAL SPOT prices for BTC, ETH, SOL, XRP, MATIC
# Utilizes the function "get_historial_data" that 
# Takes one (1) parameter ("futures_ticker") as a (str)

btc_spot_historical = get_historical_data("BTC/USD")
eth_spot_historical = get_historical_data("ETH/USD")
sol_spot_historical = get_historical_data("SOL/USD")
xrp_spot_historical = get_historical_data("XRP/USD")
matic_spot_historical = get_historical_data("MATIC/USD")


BTC/USD: Pulling Historical Price Data...
ETH/USD: Pulling Historical Price Data...
SOL/USD: Pulling Historical Price Data...
XRP/USD: Pulling Historical Price Data...
MATIC/USD: Pulling Historical Price Data...


In [6]:
# Combines all the HISTORICAL SPOT price data into a single DataFrame

historical_spot_prices = pd.concat(
    [btc_spot_historical, 
     eth_spot_historical, 
     sol_spot_historical, 
     xrp_spot_historical, 
     matic_spot_historical], 
    axis="columns", join="inner")
display(historical_spot_prices)


,BTC/USD,ETH/USD,SOL/USD,XRP/USD,MATIC/USD
Date,,,,,
2021-12-09 13:00:00,49377.0,4302.1,189.1000,0.917700,2.250257
2021-12-09 14:00:00,48602.0,4237.0,184.7375,0.890925,2.177543
2021-12-09 15:00:00,48496.0,4204.2,185.0550,0.891900,2.191182
2021-12-09 16:00:00,48548.0,4208.2,183.9875,0.896800,2.198190
2021-12-09 17:00:00,48207.0,4164.9,182.3925,0.885550,2.131743
...,...,...,...,...,...
2022-02-09 20:00:00,44802.0,3261.6,115.8525,0.896625,2.022768
2022-02-09 21:00:00,44491.0,3238.5,114.5925,0.892850,2.021084
2022-02-09 22:00:00,44603.0,3253.8,115.0325,0.897150,2.035239


In [7]:
# Pulls HISTORICAL FUTURES price data for BTC, ETH, SOL, XRP, MATIC
# Utilizes the function "get_historial_data" that 
# Takes one (1) parameter ("futures_ticker") as a (str)

btc_futures_historical = get_historical_data("BTC-PERP")
eth_futures_historical = get_historical_data("ETH-PERP")
sol_futures_historical = get_historical_data("SOL-PERP")
xrp_futures_historical = get_historical_data("XRP-PERP")
matic_futures_historical = get_historical_data("MATIC-PERP")


BTC-PERP: Pulling Historical Price Data...
ETH-PERP: Pulling Historical Price Data...
SOL-PERP: Pulling Historical Price Data...
XRP-PERP: Pulling Historical Price Data...
MATIC-PERP: Pulling Historical Price Data...


In [8]:
# Combines all the HISTORICAL FUTURES price data into a single DataFrame

historical_futures_prices = pd.concat(
    [btc_futures_historical, 
     eth_futures_historical, 
     sol_futures_historical, 
     xrp_futures_historical, 
     matic_futures_historical], 
    axis="columns", join="inner")

display(historical_futures_prices)


,BTC-PERP,ETH-PERP,SOL-PERP,XRP-PERP,MATIC-PERP
Date,,,,,
2021-12-09 13:00:00,49378.0,4303.5,189.1275,0.917475,2.251595
2021-12-09 14:00:00,48612.0,4237.3,184.7325,0.890725,2.177235
2021-12-09 15:00:00,48498.0,4202.9,185.0500,0.892125,2.191510
2021-12-09 16:00:00,48552.0,4207.0,184.0050,0.896425,2.199933
2021-12-09 17:00:00,48211.0,4165.8,182.3975,0.884950,2.133625
...,...,...,...,...,...
2022-02-09 20:00:00,44809.0,3260.5,115.8650,0.896775,2.020895
2022-02-09 21:00:00,44498.0,3238.7,114.5800,0.892425,2.019697
2022-02-09 22:00:00,44597.0,3254.6,115.0325,0.897450,2.035610


In [9]:
# Bitcoin Futures & Spot Price DataFrame
btc_prices = pd.concat([btc_futures_historical, btc_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in BTC, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
btc_prices["Arbitrage Spread"] = btc_prices["BTC-PERP"] - btc_prices["BTC/USD"]
btc_prices["Arbitrage Spread as % of Spot Price"] = btc_prices["Arbitrage Spread"] / btc_prices["BTC/USD"]
btc_prices["30-Day SMA of Spread in BTC"] = btc_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

btc_prices


,BTC-PERP,BTC/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in BTC
Date,,,,,
2021-12-09 13:00:00,49378.0,49377.0,1.0,0.000020,NaN
2021-12-09 14:00:00,48612.0,48602.0,10.0,0.000206,NaN
2021-12-09 15:00:00,48498.0,48496.0,2.0,0.000041,NaN
2021-12-09 16:00:00,48552.0,48548.0,4.0,0.000082,NaN
2021-12-09 17:00:00,48211.0,48207.0,4.0,0.000083,NaN
...,...,...,...,...,...
2022-02-09 20:00:00,44809.0,44802.0,7.0,0.000156,1.833333
2022-02-09 21:00:00,44498.0,44491.0,7.0,0.000157,1.866667
2022-02-09 22:00:00,44597.0,44603.0,-6.0,-0.000135,1.633333


In [10]:
# Bitcoin Futures vs Spot Price Chart

btc_prices_plot = btc_prices.hvplot.line(
    x="Date", y= ["BTC-PERP", "BTC/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Bitcoin Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Bitcoin Futures & Spot Markets")
btc_prices_plot


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [11]:
# Bitcoin Arbitrage Spread in BTC (Visualization)

btc_arbitrage_plot = btc_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread",
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread",
    legend='top_left',
    title = "Bitcoin Arbitrage Spread: Futures vs Spot Market Prices")
btc_arbitrage_plot


:Curve   [Date]   (Arbitrage Spread)

In [12]:
# Bitcoin 30-Day Simple Moving Average Arbitrage Spread (Visualization)

btc_30d_arbitrage_plot = btc_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in BTC",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA Arbitrage Spread in BTC",
    legend='top_left',
    title = "Bitcoin Arbitrage Spread: 30 Day Simple Moving Average")

btc_30d_arbitrage_plot

:Curve   [Date]   (30-Day SMA of Spread in BTC)

In [13]:
# Ethereum Futures & Spot Price DataFrame
eth_prices = pd.concat([eth_futures_historical, eth_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in ETH, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
eth_prices["Arbitrage Spread"] = eth_prices["ETH-PERP"] - eth_prices["ETH/USD"]
eth_prices["Arbitrage Spread as % of Spot Price"] = eth_prices["Arbitrage Spread"] / eth_prices["ETH/USD"]
eth_prices["30-Day SMA of Spread in ETH"] = eth_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()
eth_prices


,ETH-PERP,ETH/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in ETH
Date,,,,,
2021-12-09 13:00:00,4303.5,4302.1,1.4,0.000325,NaN
2021-12-09 14:00:00,4237.3,4237.0,0.3,0.000071,NaN
2021-12-09 15:00:00,4202.9,4204.2,-1.3,-0.000309,NaN
2021-12-09 16:00:00,4207.0,4208.2,-1.2,-0.000285,NaN
2021-12-09 17:00:00,4165.8,4164.9,0.9,0.000216,NaN
...,...,...,...,...,...
2022-02-09 20:00:00,3260.5,3261.6,-1.1,-0.000337,-0.273333
2022-02-09 21:00:00,3238.7,3238.5,0.2,0.000062,-0.236667
2022-02-09 22:00:00,3254.6,3253.8,0.8,0.000246,-0.200000


In [14]:
# Ethereum Futures vs Spot Price Chart

eth_prices_plot = eth_prices.hvplot.line(
    x="Date", y= ["ETH-PERP", "ETH/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Ethereum Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Ethereum Futures & Spot Markets")
eth_prices_plot


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [15]:
# Ethereum Arbitrage Spread Chart

eth_arbitrage_plot = eth_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread",
    height=400, width=1000,
    xlabel='Date', ylabel = "Aritrage Spread",
    legend='top_left',
    title = "Ethereum Arbitrage Spread: Futures vs Spot Market Prices")

eth_arbitrage_plot

:Curve   [Date]   (Arbitrage Spread)

In [16]:
# Ethereum 30-Day Simple Moving Average Arbitrage Spread Chart

eth_30d_arbitrage_plot = eth_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in ETH",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in ETH",
    legend='top_left',
    title = "Ethereum Arbitrage Spread: 30 Day Simple Moving Average")

eth_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in ETH)

In [17]:
# Solana Futures & Spot Price DataFrame concatenation 
sol_prices = pd.concat([sol_futures_historical, sol_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in SOL, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
sol_prices["Arbitrage Spread"] = sol_prices["SOL-PERP"] - sol_prices["SOL/USD"]
sol_prices["Arbitrage Spread as % of Spot Price"] = sol_prices["Arbitrage Spread"] / sol_prices["SOL/USD"]
sol_prices["30-Day SMA of Spread in SOL"] = sol_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

sol_prices


,SOL-PERP,SOL/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in SOL
Date,,,,,
2021-12-09 13:00:00,189.1275,189.1000,0.0275,0.000145,NaN
2021-12-09 14:00:00,184.7325,184.7375,-0.0050,-0.000027,NaN
2021-12-09 15:00:00,185.0500,185.0550,-0.0050,-0.000027,NaN
2021-12-09 16:00:00,184.0050,183.9875,0.0175,0.000095,NaN
2021-12-09 17:00:00,182.3975,182.3925,0.0050,0.000027,NaN
...,...,...,...,...,...
2022-02-09 20:00:00,115.8650,115.8525,0.0125,0.000108,0.005250
2022-02-09 21:00:00,114.5800,114.5925,-0.0125,-0.000109,0.004583
2022-02-09 22:00:00,115.0325,115.0325,0.0000,0.000000,0.000667


In [18]:
# Solana Futures vs Spot Price Chart

sol_prices_plot = sol_prices.hvplot.line(
    x="Date", y= ["SOL-PERP", "SOL/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Solana Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Solana Futures & Spot Markets")

sol_prices_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [19]:
# Solana Arbitrage Spread Chart

sol_arbitrage_plot = sol_prices.hvplot.line(
    x="Date", y= ["Arbitrage Spread"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread",
    legend='top_left',
    title = "Solana Arbitrage Spread: Futures vs Spot Market Prices")

sol_arbitrage_plot

:Curve   [Date]   (Arbitrage Spread)

In [20]:
# Solana 30-Day Simple Moving Average Arbitrage Spread Chart

sol_30d_arbitrage_plot = sol_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in SOL",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in SOL",
    legend='top_left',
    title = "Solana Arbitrage Spread: 30-Day Simple Moving Average")

sol_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in SOL)

In [21]:
# Ripple Futures & Spot Price DataFrame concatenation
xrp_prices = pd.concat([xrp_futures_historical, xrp_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in XRP, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
xrp_prices["Arbitrage Spread"] = xrp_prices["XRP-PERP"] - xrp_prices["XRP/USD"]
xrp_prices["Arbitrage Spread as % of Spot Price"] = xrp_prices["Arbitrage Spread"] / xrp_prices["XRP/USD"]
xrp_prices["30-Day SMA of Spread in XRP"] = xrp_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

xrp_prices


,XRP-PERP,XRP/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in XRP
Date,,,,,
2021-12-09 13:00:00,0.917475,0.917700,-0.000225,-0.000245,NaN
2021-12-09 14:00:00,0.890725,0.890925,-0.000200,-0.000224,NaN
2021-12-09 15:00:00,0.892125,0.891900,0.000225,0.000252,NaN
2021-12-09 16:00:00,0.896425,0.896800,-0.000375,-0.000418,NaN
2021-12-09 17:00:00,0.884950,0.885550,-0.000600,-0.000678,NaN
...,...,...,...,...,...
2022-02-09 20:00:00,0.896775,0.896625,0.000150,0.000167,0.000127
2022-02-09 21:00:00,0.892425,0.892850,-0.000425,-0.000476,0.000097
2022-02-09 22:00:00,0.897450,0.897150,0.000300,0.000334,0.000096


In [22]:
# Ripple Futures vs Spot Price Chart

xrp_prices_plot = xrp_prices.hvplot.line(
    x="Date", y= ["XRP-PERP", "XRP/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Ripple Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Ripple Futures & Spot Markets")

xrp_prices_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [23]:
# Ripple Arbitrage Spread Chart

xrp_arbitrage_plot = xrp_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread",
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread",
    legend='top_left',
    title = "Ripple Arbitrage Spread: Futures vs Spot Market Prices")

xrp_arbitrage_plot


:Curve   [Date]   (Arbitrage Spread)

In [24]:
# Ripple 30-Day Simple Moving Average Arbitrage Spread Chart

xrp_30d_arbitrage_plot = xrp_prices.hvplot.line(
    x="Date", y= "30-Day SMA of Spread in XRP",
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in XRP",
    legend='top_left',
    title = "Ripple Arbitrage Spread: 30-Day Simple Moving Average")

xrp_30d_arbitrage_plot


:Curve   [Date]   (30-Day SMA of Spread in XRP)

In [25]:
# Polygon Futures & Spot Price DataFrame concatenation

matic_prices = pd.concat([matic_futures_historical, matic_spot_historical], axis="columns", join="inner")

# Calculates the Arbitrage Spread in MATIC, Arbitrage Spread as % of Spot Price, and 30-Day Simple Moving Average (SMA)
matic_prices["Arbitrage Spread"] = matic_prices["MATIC-PERP"] - matic_prices["MATIC/USD"]
matic_prices["Arbitrage Spread as % of Spot Price"] = matic_prices["Arbitrage Spread"] / matic_prices["MATIC/USD"]
matic_prices["30-Day SMA of Spread in MATIC"] = matic_prices["Arbitrage Spread"].rolling(window=30).mean().dropna()

matic_prices


,MATIC-PERP,MATIC/USD,Arbitrage Spread,Arbitrage Spread as % of Spot Price,30-Day SMA of Spread in MATIC
Date,,,,,
2021-12-09 13:00:00,2.251595,2.250257,0.001338,0.000595,NaN
2021-12-09 14:00:00,2.177235,2.177543,-0.000308,-0.000141,NaN
2021-12-09 15:00:00,2.191510,2.191182,0.000328,0.000150,NaN
2021-12-09 16:00:00,2.199933,2.198190,0.001743,0.000793,NaN
2021-12-09 17:00:00,2.133625,2.131743,0.001882,0.000883,NaN
...,...,...,...,...,...
2022-02-09 20:00:00,2.020895,2.022768,-0.001873,-0.000926,-0.000974
2022-02-09 21:00:00,2.019697,2.021084,-0.001387,-0.000686,-0.001004
2022-02-09 22:00:00,2.035610,2.035239,0.000371,0.000182,-0.000978


In [26]:
# Polygon Futures vs Spot Price Chart

matic_prices_plot = matic_prices.hvplot.line(
    x="Date", y= ["MATIC-PERP", "MATIC/USD"],
    height=400, width=1000,
    xlabel='Date', ylabel = "Polygon Futures / Spot Price",
    legend='top_left',
    title = "Historical Prices for Polygon Futures & Spot Markets")

matic_prices_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [27]:
# Polygon Arbitrage Spread Chart

matic_arbitrage_plot = matic_prices.hvplot.line(
    x="Date", y= "Arbitrage Spread",
    height=400, width=1000,
    xlabel='Date', ylabel = "Arbitrage Spread",
    legend='top_left',
    title = "Polygon Arbitrage Spread: Futures vs Spot Market Prices")

matic_arbitrage_plot

:Curve   [Date]   (Arbitrage Spread)

In [28]:
# Polygon 30-Day Simple Moving Average Arbitrage Spread Chart

matic_arbitrage_plot = matic_prices.hvplot.line(
    x="Date", y= ["30-Day SMA of Spread in MATIC"],
    height=400, width=1000,
    xlabel='Date', ylabel = "30-Day SMA of Spread in MATIC",
    legend='top_left',
    title = "Polygon Arbitrage Spread: 30-Day Simple Moving Average")

matic_arbitrage_plot

:Curve   [Date]   (30-Day SMA of Spread in MATIC)

In [29]:
# Function to "get_summary_stats" for each asset 
# Function takes three (3) parameters (asset_dataframe, spot_ticker, asset_name)

btc_summary = get_summary_stats(btc_prices, "BTC/USD", "Bitcoin")
eth_summary = get_summary_stats(eth_prices, "ETH/USD", "Ethereum")
sol_summary = get_summary_stats(sol_prices, "SOL/USD", "Solana")
xrp_summary = get_summary_stats(xrp_prices, "XRP/USD", "Ripple")
matic_summary = get_summary_stats(matic_prices, "MATIC/USD", "Polygon")


In [30]:
# Summary statistics for all asset combine into a single DataFrame
arbitrage_summary_statistics = pd.concat([btc_summary, eth_summary, sol_summary, xrp_summary, matic_summary], axis="columns", join="inner")

# Chart shows arbitrage spreads as a percentrage of the asset's spot price for comparison
arbitrage_summary_statistics

,Bitcoin,Ethereum,Solana,Ripple,Polygon
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.000094,0.000048,0.000050,-0.000097,-0.000240
std,0.000202,0.000238,0.000315,0.000674,0.000500
min,-0.000905,-0.001086,-0.001864,-0.010107,-0.002651
25%,-0.000046,-0.000114,-0.000127,-0.000245,-0.000543
50%,0.000087,0.000032,0.000054,0.000000,-0.000195
75%,0.000232,0.000206,0.000244,0.000196,0.000100
max,0.000870,0.000898,0.002018,0.002697,0.001571


In [31]:
# Function to "get_average_spread" for each asset over entire period

btc_average_spread = get_average_spread(btc_prices, "BTC/USD", "Bitcoin")
eth_average_spread = get_average_spread(eth_prices, "ETH/USD", "Ethereum")
sol_average_spread = get_average_spread(sol_prices, "SOL/USD", "Solana")
xrp_average_spread = get_average_spread(xrp_prices, "XRP/USD", "Ripple")
matic_average_spread = get_average_spread(matic_prices, "MATIC/USD", "Polygon")


# Combines data into a single DataFrame
average_spreads = pd.concat(
    [btc_average_spread, 
     eth_average_spread, 
     sol_average_spread, 
     xrp_average_spread, 
     matic_average_spread], 
    axis="rows", join="inner")

# Bar Chart of Historical Arbitrage Spreads per Day on Average
spreads_bar_chart = average_spreads.hvplot.bar(
    title='Average Daily Arbitrage Spreads as % of Spot Price'
    ).opts(
    yformatter='%.5f',
    hover_color='yellow')

display(spreads_bar_chart)


:Bars   [index]   (Historical Daily Arbitrage Spreads)